In [13]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [25]:

club = "computer"
campus = "Columbus"

d = requests.get(
    #f'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&s={club}&c={campus}'
    'https://activities.osu.edu/involvement/student_organizations/find_a_student_org?v=list&l=ALL&c=Columbus')
data = d.text
# print(data)


In [36]:

soup = BeautifulSoup(data, 'html.parser')

# Good tutorial:
#https://medium.com/geekculture/web-scraping-tables-in-python-using-beautiful-soup-8bbc31c5803e

# Finding the holy table
table = soup.find('table', class_='c-table')
# print(table)
# Collecting Ddata
df = pd.DataFrame(columns=["name", "statement", "topics"])
# df = pd.DataFrame()
for row in table.find_all('tr')[1:]:
    # print(f"Our row is {row}")
    # Find all data for each column
    columns = row.find_all('td')
    club_name = columns[0].text.strip()
    statement = columns[1].text.strip()
    topics = columns[2].text.strip().split(",")
    if topics[-1] == '':
        topics.pop()
    datarow = pd.DataFrame([{"name":club_name, "statement":statement, "topics": topics}])
    df = df.append(datarow, ignore_index=True)
df

,name,statement,topics
0,"1 Day For The K.I.A., Inc.",1 Day for the K.I.A.’s purpose is to raise awa...,"[Columbus, Awareness/Activism, Community Ser..."
1,1825 at Ohio State,To help students at The Ohio State University ...,"[Columbus, Religious/Spiritual, Awareness/Ac..."
2,1girl,We strive to provide girls and nonbinary stude...,"[Columbus, Community Service/Service Learning..."
3,2022 Intercollegiate Taiwanese American Studen...,The purpose of this organization is to unite O...,"[Columbus, Ethnic/Cultural]"
4,The 2nd and 7 Foundation Medical Branch,The mission of the 2nd and 7 Foundation Medica...,"[Columbus, Community Service/Service Learning..."
...,...,...,...
1764,"Zeta Phi Beta Sorority, Incorporated","To foster the ideas of service, charity, schol...","[Columbus, Social Fraternities/Sororities, E..."
1765,Zeta Tau Alpha,The purpose of Zeta Tau Alpha is the intensify...,"[Columbus, Social Fraternities/Sororities, A..."
1766,"Zoo, Wildlife, and Conservation Medicine Club","The purpose of the OSU CVM Zoo, Wildlife, and ...","[Columbus, Academic/College, Special Interest]"
1767,Zoology Club at The Ohio State University,The purpose of Zoology Club is to enhance the ...,"[Columbus, Academic/College, Special Interest]"
